In [5]:
from utils.models import runfullseasonpredictor

In [9]:
regressorlist = ["SVR"]
eon = 0.15

In [10]:
runfullseasonpredictor(regressorlist, eon)

SVR


NameError: name 'estimatororneigbor' is not defined